- all the input data has to be lowercase for the embeddings, it does not recoginize "The"
- only 20 unique sentences, they go in a loop.
- construct your own dataset from some book


In [1]:
# download dataset:
import torchtext.vocab as vocab

def load_glove_embeddings(dim):
    # Loads the specified GloVe embeddings
    glove = vocab.GloVe(name='6B', dim=dim)
    return glove

# Specify the desired dimension
embed_dim = 100  # For 100-dimensional embeddings
glove_embeddings = load_glove_embeddings(embed_dim)

# Now, `glove_embeddings` holds the loaded embeddings
print(f"Loaded {len(glove_embeddings.stoi)} words with {embed_dim}-dimensional embeddings.")


.vector_cache/glove.6B.zip: 862MB [02:41, 5.34MB/s]                                                                    
100%|██████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:06<00:00, 63313.32it/s]


Loaded 400000 words with 100-dimensional embeddings.


In [2]:
import pandas as pd

# Enhanced dataset with more variety
sentences = [
    "The cat sat on the", "The dog ran around the", "The sun was shining in the",
    "The baby laughed at the", "The teacher wrote on the", "A car drove over the",
    "He opened the", "She closed the", "The player won the", "The artist drew a",
    "The girl danced in the", "The boy played in the", "The wind blew through the",
    "The author discussed the", "The scientist discovered a", "The historian studied the",
    "The chef cooked a", "The farmer planted a", "The journalist wrote about the",
    "The programmer debugged the"
]
next_words = [
    "mat", "block", "sky", "toy", "board", "bridge", "door", "window", "game", "portrait",
    "room", "yard", "trees", "topic", "method", "artifact", "dish", "seed", "event", "program"
]

df = pd.DataFrame({"sentence": sentences, "next_word": next_words})


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
# Load GloVe Embeddings
glove_file = "./.vector_cache/glove.6B.100d.txt"
glove_embeddings = {}
with open(glove_file, "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = torch.tensor([float(x) for x in values[1:]], dtype=torch.float32)
        glove_embeddings[word] = vector

In [5]:
import pandas as pd

df_json = pd.read_json('./dataset_rnn.json')
# turn into dataframe make the right columns
df = pd.DataFrame( columns=['sentence', 'next_word'])
for i in range(len(df_json['data'])):
    # no append
    df.loc[i] = [df_json['data'][i]['sentence'].lower(), df_json['data'][i]['next_word'].lower()]
'''
This is only an example. Change as you see fit. Right now there is not split between train and test data. You will have to implement that.
'''    
# Assuming glove_embeddings has been loaded as shown earlier
def sentence_to_embedding(sentence):
    words = sentence.split()
    embeddings = [glove_embeddings.get(word, torch.zeros(100, dtype=torch.float32)) for word in words]
    return torch.stack(embeddings)

class SentenceDataset(Dataset):
    def __init__(self, df):
        self.df = df
        next_words = df["next_word"].tolist()
        self.word_to_idx = {word: i for i, word in enumerate(sorted(set(next_words)))}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sentence = self.df.iloc[idx, 0] 
        next_word = self.df.iloc[idx, 1] 
        embedding = sentence_to_embedding(sentence)
        next_word_idx = self.word_to_idx[next_word]
        return {"sentence": embedding, "next_word": next_word_idx}

In [6]:
# shuffle data randomly
# split into 0.7, 0.15, 0.15
# evaluate using k-fold cross evaluation

# df = df.sample(frac=1, random_state=42).reset_index(drop=True)
data_len = len(df_json['data'])
train_split = int(data_len * 0.7)
test_split = train_split + int(data_len * 0.15)

train_df = df.iloc[:train_split]
test_df = df.iloc[train_split:test_split]
val_df = df.iloc[test_split:]

train_dataset = SentenceDataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataset = SentenceDataset(test_df)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
val_dataset = SentenceDataset(val_df)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim  # Save hidden_dim as an instance variable
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden and cell states with dimensions: (num_layers, batch_size, hidden_dim)
        h0 = torch.zeros(1, x.size(0), self.hidden_dim)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim)
        # Forward pass through LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        # Pass the output of the last time step to the fully connected layer
        out = self.fc(out[:, -1, :])
        return out


output_dim = len(train_dataset.word_to_idx)  # Number of unique next words
model = LSTMModel(input_dim=100, hidden_dim=128, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def calculate_perplexity(loss):
    return torch.exp(loss)

# Training Loop with Perplexity Calculation
for epoch in range(10):
    total_loss = 0
    for batch in train_dataloader:
        model.train()
        sentences = batch["sentence"]
        next_words = batch["next_word"]
        optimizer.zero_grad()
        outputs = model(sentences)
        loss = criterion(outputs, next_words)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_dataloader)
    perplexity = calculate_perplexity(torch.tensor(avg_loss))
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Perplexity: {perplexity.item():.4f}")


/home/piragi/projects/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1, Loss: 2.7287, Perplexity: 15.3128
Epoch 2, Loss: 0.7985, Perplexity: 2.2222
Epoch 3, Loss: 0.1439, Perplexity: 1.1548
Epoch 4, Loss: 0.0530, Perplexity: 1.0544
Epoch 5, Loss: 0.0725, Perplexity: 1.0752
Epoch 6, Loss: 0.0168, Perplexity: 1.0170
Epoch 7, Loss: 0.0114, Perplexity: 1.0114
Epoch 8, Loss: 0.0086, Perplexity: 1.0086
Epoch 9, Loss: 0.0068, Perplexity: 1.0068
Epoch 10, Loss: 0.0056, Perplexity: 1.0056


In [11]:
# check the model
model.eval()
sentence = "The programmer debugged the"
sentence_embedding = sentence_to_embedding(sentence).unsqueeze(0)
output = model(sentence_embedding)
_, predicted_idx = torch.max(output, 1)
predicted_word = list(train_dataset.word_to_idx.keys())[predicted_idx.item()]
print(f"Predicted next word: {predicted_word}")

Predicted next word: program


In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
# Load GloVe Embeddings
glove_file = "glove.6B.100d.txt"
glove_embeddings = {}
with open(glove_file, "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = torch.tensor([float(x) for x in values[1:]], dtype=torch.float32)
        glove_embeddings[word] = vector
import pandas as pd
df_json = pd.read_json('./dataset_rnn.json')
# turn into dataframe make the right columns
df = pd.DataFrame( columns=['sentence', 'next_word'])
for i in range(len(df_json['data'])):
    # no append
    df.loc[i] = [df_json['data'][i]['sentence'], df_json['data'][i]['next_word']]
'''
This is only an example. Change as you see fit. Right now there is not split between train and test data. You will have to implement that.
'''    

# Assuming glove_embeddings has been loaded as shown earlier
def sentence_to_embedding(sentence):
    words = sentence.split()
    embeddings = [glove_embeddings.get(word, torch.zeros(100, dtype=torch.float32)) for word in words]
    return torch.stack(embeddings)

class SentenceDataset(Dataset):
    def __init__(self, df):
        self.df = df
        next_words = df["next_word"].tolist()
        self.word_to_idx = {word: i for i, word in enumerate(sorted(set(next_words)))}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sentence = self.df.iloc[idx, 0]
        next_word = self.df.iloc[idx, 1]
        embedding = sentence_to_embedding(sentence)
        next_word_idx = self.word_to_idx[next_word]
        return {"sentence": embedding, "next_word": next_word_idx}

dataset = SentenceDataset(df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim  # Save hidden_dim as an instance variable
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden and cell states with dimensions: (num_layers, batch_size, hidden_dim)
        h0 = torch.zeros(1, x.size(0), self.hidden_dim)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim)
        # Forward pass through LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        # Pass the output of the last time step to the fully connected layer
        out = self.fc(out[:, -1, :])
        return out


output_dim = len(dataset.word_to_idx)  # Number of unique next words
model = LSTMModel(input_dim=100, hidden_dim=128, output_dim=output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def calculate_perplexity(loss):
    return torch.exp(loss)

# Training Loop with Perplexity Calculation
for epoch in range(10):
    total_loss = 0
    for batch in dataloader:
        model.train()
        sentences = batch["sentence"]
        next_words = batch["next_word"]
        optimizer.zero_grad()
        outputs = model(sentences)
        loss = criterion(outputs, next_words)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    perplexity = calculate_perplexity(torch.tensor(avg_loss))
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Perplexity: {perplexity.item():.4f}")


Epoch 1, Loss: 2.1249, Perplexity: 8.3717
Epoch 2, Loss: 0.3114, Perplexity: 1.3653
Epoch 3, Loss: 0.0739, Perplexity: 1.0767
Epoch 4, Loss: 0.0269, Perplexity: 1.0273
Epoch 5, Loss: 0.0136, Perplexity: 1.0137
Epoch 6, Loss: 0.0086, Perplexity: 1.0086
Epoch 7, Loss: 0.0062, Perplexity: 1.0062
Epoch 8, Loss: 0.0048, Perplexity: 1.0048
Epoch 9, Loss: 0.0038, Perplexity: 1.0038
Epoch 10, Loss: 0.0031, Perplexity: 1.0031


In [10]:
# check the model
model.eval()
sentence = "The cat sat on the"
sentence_embedding = sentence_to_embedding(sentence).unsqueeze(0)
output = model(sentence_embedding)
_, predicted_idx = torch.max(output, 1)
predicted_word = list(dataset.word_to_idx.keys())[predicted_idx.item()]
print(f"Predicted next word: {predicted_word}")
# Output: Predicted next word: mat

NameError: name 'dataset' is not defined

In [18]:

import torch.nn.functional as F
def generate_sentence(model, start_sentence, max_length=20):
    model.eval()  # Ensure the model is in evaluation mode
    sentence = start_sentence
    for _ in range(max_length):
        output = model(sentence_to_embedding(sentence).unsqueeze(0))
        _, predicted_idx = torch.max(output, 1)
        predicted_word = list(train_dataset.word_to_idx.keys())[predicted_idx.item()]
        sentence += predicted_word

    words = sentence.split()
    return sentence

# Example usage after training
start_fragment = "The cat sat on the"
generated_sentence = generate_sentence(model, start_fragment)
#used_words = set(words)  # Keep track of words used in the sentence to apply penalties
print("Generated Sentence:", generated_sentence)



NameError: name 'dataset' is not defined